# Imports

In [ ]:
!pip install -q -U google-generativeai 
!pip install --upgrade transformers datasets[audio] accelerate 

In [ ]:
import os
import time
import numpy as np
import pandas as pd
from PIL import Image
import soundfile as sf
from json import loads
import seaborn as sns 
import matplotlib.pyplot as plt
import typing_extensions as typing
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# Data Loading

In [ ]:
df = pd.read_csv('/kaggle/input/updated-ground-truth/ground_labels_new.csv') 

df['Primary Label'] = df['Primary Label'].str.lower() 
df = df[df['Primary Label'].isin(['appropriate', 'inappropriate'])]

'''
ids_df = pd.read_csv('/kaggle/input/false-positives-prompt-a/gemini-testing-prompt-a-revised-remaining.csv') 
ids_df['Video Id'] = ids_df['Video Id'].str.lstrip("'") 
ids_ = ids_df['Video Id'].to_list() 
df = df[df['Video Id'].isin(ids_)] 
'''

df.head()

In [ ]:
# class distribution 

df['Primary Label'].value_counts() 

# Extracting video ids and primary labels 

video_ids = list(df['Video Id']) 
primary_labels = list(df['Primary Label']) 

In [ ]:
# checking how many are available 

available_ad_ids = os.listdir('/kaggle/input/youtube-data/Ads/Ads') 
len(available_ad_ids) 

count = 0 
for x in video_ids: 
    if x in available_ad_ids: 
        count += 1 
    else: 
        print(count)    # this id is not available 

count 

**Defining Model** 

In [ ]:
import google.generativeai as genai 
os.environ["API_KEY"] = ""
genai.configure(api_key=os.environ["API_KEY"]) 

In [ ]:
model = genai.GenerativeModel("gemini-2.5-flash", system_instruction="You are an expert content moderator.") 

In [ ]:
class LLM_Output(typing.TypedDict):
    label: str
    language: list[str]
    response: str

In [ ]:
start = 0
end = len(video_ids) 

**Prompt** 

In [ ]:
prompt =  """"A video can be considered inappropriate for children if it contains physical violence (cartoonish, realistic, or gory), interpersonal violence (bullying, pranks, meanness, belittling, controlling behavior, talking down to others, or manipulation), self-harm or suicide (depictions of harm inflicted on oneself or suicidal thoughts/tendencies), extreme stunts (life-endangering, high-risk activities/challenges that require adult supervision), dangerous products or services (like paintball, airsoft, fireworks, weapons, hunting equipment, graffiti products), scary content (horror, suspense, loud violence, zombies, skeletons, masks, scary clowns, blood, dangerous fire, car crashes, medical procedures or other scary visuals), sexual content (innuendos, sexual behavior, nudity, suggestiveness), intimacy (romantic conduct, couples expressing love, sensual contact, dating, relationships or any expression of love or loss for a significant other,  lyrics about or mentions of love or romance, heartbreak), inappropriate clothing (significant skin exposure or sheer clothing outside of appropriate contexts such as sunbathing, swinmwear,etc), substance use (mention, depictions, or use of alcohol, drugs, tobacco), gambling (betting, gambling, casinos, online or real world casino games, online trading), offensive language (from mild to severe with words such as “stupid” or “idiot”), stereotypes (racial, ethnic, gender, body shaming), crude humor (inappropriate humor, innuendos), weapons (guns, swords, knives, etc but toy guns such as nerf guns are fine), unsuitable media (ads for violent or adult-themed video games, casino games, games with romance or battle or first person-shooter games), fight sports (boxing, wrestling, martial arts), financial content (difference, rolling spot forex, financial spread betting and synonymous products but banking ads are fine), health and beauty services (cosmetic procedures, weight loss, tanning, piercings, tattoos, cosmetics and makeup (no matter their essential benefits) and other products related to external personal care focused on body image but ads on fashion products and essential hygiene products like sunscreen, shampoo, toothpaste, etc. are fine, however, products like sanitary care and hair removal is inappropriate), ads or sponsored content for consumable food or drink products (regardless of nutritional content), where the product is consumed by a person or shown with branding, packaging, or promotional intent are inappropriate; general content like cooking shows, recipes, or scenes where food is simply present or being eaten without promotion are acceptable, pet/animal food products and healthcare, healthcare or medical topics (reproductive health, mental health, substance abuse, eating disorders, pharmaceuticals, health insurance, any nutritional or dietary supplements), sensitive topics (politics, religion, controversial social issues, astrology, occult, paranormal, religious sermons), privacy or safety risks (services or competitions requiring personal information, social networks for strangers, spy services, or video game skin trading), or any explicit depictions or discussions of death or dying. Consider that you are an expert content moderator. Based on the provided audio, is the video advertisement using this auido inappropriate for children on this definition? If it is, label it as “inappropriate.” If it is not, label it as “appropriate.” Justify your conclusion. Also, mention what language is being spoken if there is any. 
Make the outputs in JSON format: {“label”: “appropriate” or “inappropriate”, “justification”: “brief explanation of the classification”, “language”: “language detected in the video”}."""
prompt 

In [ ]:
len(video_ids) 

**Running Model on Dataset** 

In [ ]:
ids = []
predicted_labels = []
languages = []
responses = []
ground_truths_ = []
remaining = []

for i in range(start, end): 

    if video_ids[i] in available_ad_ids:

        try:
            contents_of_ad = os.listdir('/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i]) 
            if 'audio.mp3' not in contents_of_ad:
                print(f"audio.mp3 not found for {video_ids[i]}")
                remaining.append(video_ids[i])
                continue

            path_to_audio = '/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i] + '/audio.mp3' 

            # Short delay before upload
            time.sleep(2)
            audio_file = genai.upload_file(path=path_to_audio)

            # Delay before calling the model
            time.sleep(3)

            try: 
                response = model.generate_content([audio_file, prompt],
                    generation_config=genai.GenerationConfig(
                        response_mime_type="application/json",
                        response_schema=LLM_Output
                    ),
                    safety_settings={
                        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
                    }
                )
            except Exception as e:
                print(f"Error making inference: {e}")
                remaining.append(video_ids[i])
                continue 

            # Delay to reduce risk of rapid-response processing
            time.sleep(1)

            try:
                print("Completed for video number:", i, ' ', video_ids[i])
                dictionary = loads(response.text)
                print('True Label:', primary_labels[i], 'Response:', dictionary)

                ids.append(video_ids[i])
                predicted_labels.append(dictionary['label'])
                languages.append(dictionary['language'])
                responses.append(dictionary['response'])
                ground_truths_.append(primary_labels[i]) 
                time.sleep(30)
                
            except Exception as e:
                print(f"Error processing response.text: {e}")
                remaining.append(video_ids[i])
                time.sleep(30)
                continue

        except Exception as e:
            print(f"Unexpected error: {e}")
            remaining.append(video_ids[i])
            time.sleep(30)

In [ ]:
# At the end, print remaining videos 

remaining_df = pd.DataFrame({'Video Id': remaining})

# Save to CSV
remaining_df.to_csv(f'remaining-{start}-{end}.csv', index=False)

print("Remaining videos saved to remaining.csv")
print("Remaining videos with errors:", remaining) 

In [ ]:
for i in range(len(responses)): 
    print('True Label: ', ground_truths_[i], '\tPrediction: ', predicted_labels[i]) 

In [ ]:
new_df = pd.DataFrame({
    'Video Id': ids,
    'Primary Label': ground_truths_,
    'Predicted Label': predicted_labels,
    'Response': responses, 
    'Languages': languages 
})

new_df.head() 

In [ ]:
new_df.to_csv(f'/kaggle/working/gemini-only-audios-{start}-{end}.csv', index=False)

In [ ]:
# Changing to binary lists 

predictions = [1 if pred == 'inappropriate' else 0 for pred in predicted_labels] 
ground_truths = [1 if label == 'inappropriate' else 0 for label in ground_truths_] 

In [ ]:
# Obtaining classification report 
from sklearn.metrics import classification_report 

report = classification_report(ground_truths, predictions) 
print(report) 

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ground_truths, predictions)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Appropriate', 'Inapproriate'], yticklabels=['Appropriate', 'Inapproriate'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()